In [118]:
import real_estate_api as rea

from typing import Union, List
from types import ModuleType
from beartype._decor.decormain import beartype
import warnings

import numpy as np
import pandas as pd
import sys
import importlib

from itertools import chain
import random

from sklearn.base import TransformerMixin, BaseEstimator, RegressorMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

import keys as k
keys = k.getKeys()

In [2]:
user_home = rea.get_UserHome(keys['SampleHouse'].strip("\'"))
hoi = rea.get_HousesOfInterest(user_home, n=2000, listed_to_sold_ratio=0.3, verbose=True)
gd = rea.geo_data(hoi['geo'])

Shortfall in listed houses detected, appending 114 of current listing to results.


In [3]:
# Can this be the first step in the pipeline or no?
fg = rea.FeatureGenerator(
    houses = hoi['houses'], 
    gd=gd,
    user_home=user_home
)

In [4]:
class ToDataFrame(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return pd.DataFrame(X).drop_duplicates(subset=['Property_ID'])

pipe = Pipeline([
    ('format_data', ToDataFrame())
])

output = pipe.fit_transform(fg.features, fg.targets)
output

,Property_ID,Days_listed,Days_updated,baths_full,baths_3qtr,baths_half,baths_1qtr,year_built,lot_sqft,sqft,garage,stories,beds,tags,new_construction,distance_to_home,angle_from_home
0,2713193735,0,0,6,1,0,0,1930,34665,5690,2,1,7,"[city_view, community_outdoor_space, den_or_of...",False,6.956224,2.511557
1,2659298958,0,0,1,1,0,0,1979,6500,1660,1,1,5,"[community_outdoor_space, den_or_office, dinin...",False,10.862586,1.688721
2,1093027260,1,1,2,1,1,0,2012,1975,1780,1,1,3,"[city_view, dining_room, dishwasher, energy_ef...",False,1.047090,1.788632
3,1648537949,1,1,2,0,0,0,1962,7618,3310,2,1,6,"[community_outdoor_space, dining_room, family_...",False,0.179029,0.598855
4,1757015623,1,1,3,2,0,0,1911,3376,2800,0,1,5,"[dining_room, forced_air, hardwood_floors, oce...",False,6.660062,2.158812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1808,1627256650,0,128,1,1,1,0,1928,4452,2470,0,1,3,"[central_air, city_view, community_outdoor_spa...",False,3.265067,-3.113212
1809,1311822489,0,128,3,0,0,0,1953,8037,3610,2,1,4,"[carport, central_air, community_outdoor_space...",False,6.987278,1.259727
1810,2102420682,0,128,1,1,0,0,1947,6395,2284,0,1,3,"[community_outdoor_space, den_or_office, dinin...",False,4.747036,-2.757511
1811,2141376707,0,124,1,1,1,0,1945,5000,2380,1,1,4,"[community_clubhouse, community_gym, community...",False,6.023588,1.357392


In [ ]:
to_data_frame = ToDataFrame()
X_t = to_data_frame.fit_transform(fg.features)
X_t

,Days_listed,Days_updated,baths_full,baths_3qtr,baths_half,baths_1qtr,year_built,lot_sqft,sqft,garage,stories,beds,type,tags,new_construction,distance_to_home
0,1,1,1,1,0,0,1942,7000,1992,1,1,4,single_family,"[community_outdoor_space, den_or_office, dinin...",False,0.087471
1,1,1,1,0,1,0,1988,1071,1300,0,1,2,single_family,"[city_view, community_outdoor_space, community...",False,0.101667
2,2,2,1,1,0,0,1913,3500,1750,0,1,3,single_family,"[community_security_features, den_or_office, d...",False,0.205025
3,2,2,1,1,1,0,1980,8100,3280,2,1,3,single_family,"[community_outdoor_space, dining_room, dishwas...",False,0.068888
4,2,2,1,1,1,0,1948,7680,2180,2,1,4,single_family,"[central_air, community_outdoor_space, dining_...",False,0.083503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
970,0,75,1,1,1,0,1906,4000,2260,1,1,3,single_family,"[city_view, community_outdoor_space, den_or_of...",False,0.091754
971,0,74,3,0,1,0,2006,3600,3340,2,1,5,single_family,"[central_air, community_outdoor_space, communi...",False,0.127658
972,0,73,1,1,0,0,1952,5800,2360,1,1,2,single_family,"[community_outdoor_space, community_security_f...",False,0.080205
973,0,73,1,1,0,0,1906,5700,2880,1,1,2,single_family,"[community_outdoor_space, dining_room, dishwas...",False,0.093157


In [29]:
X_t[X_t['lot_sqft'] > 5_000].sort_values('lot_sqft')

,Days_listed,Days_updated,baths_full,baths_3qtr,baths_half,baths_1qtr,year_built,lot_sqft,sqft,garage,stories,beds,type,tags,new_construction,distance_to_home
870,0,67,2,0,1,0,2004,5001,2320,2,1,4,single_family,"[community_outdoor_space, dishwasher, fireplac...",False,0.211181
225,66,6,3,0,1,0,1990,5001,3190,2,1,5,single_family,"[community_outdoor_space, den_or_office, dinin...",False,0.070469
927,0,0,4,1,0,0,2013,5002,3040,0,2,4,single_family,[],False,0.183211
327,0,10,1,1,0,0,1950,5002,2150,1,1,4,single_family,"[community_outdoor_space, dining_room, dishwas...",False,0.142171
559,0,32,2,0,0,0,1980,5005,1460,0,1,3,single_family,"[community_clubhouse, community_outdoor_space,...",False,0.145353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,53,5,0,1,0,0,1997,43560,560,0,1,2,single_family,"[city_view, community_outdoor_space, community...",False,0.102533
803,0,62,1,0,0,0,1920,43560,691,0,1,2,single_family,"[community_outdoor_space, forced_air, hardwood...",False,0.107248
85,28,5,0,1,1,0,1997,43560,1000,0,1,2,single_family,"[city_view, community_boat_facilities, communi...",False,0.101667
375,0,19,1,1,1,0,1995,43560,1954,2,1,2,single_family,"[central_air, community_security_features, dis...",False,0.117246


In [86]:
# Add more info into this for testing and observation, drop in the pipeline.
X_t['target'] = fg.targets
X_t[X_t['target'] < 300_000]

,Days_listed,Days_updated,baths_full,baths_3qtr,baths_half,baths_1qtr,year_built,lot_sqft,sqft,garage,stories,beds,type,tags,new_construction,distance_to_home,target
381,0,0,1,0,0,0,1949,7336,720,0,1,2,single_family,[],False,0.107203,250000
421,0,0,1,0,0,0,1929,1500,950,0,1,3,single_family,[],False,0.147240,167691
519,0,0,1,0,0,0,1912,8604,770,0,1,2,single_family,[],False,0.157582,250000
524,0,0,1,0,0,0,1911,3072,760,0,1,2,single_family,[],False,0.655990,250000
741,0,0,1,1,0,0,1946,6300,770,0,1,2,single_family,[],False,0.069057,170000


In [63]:
fg.houses[727].features

{'Property_ID': '1444220862',
 'Days_listed': 0,
 'Days_updated': 39,
 'baths_full': 2,
 'baths_3qtr': 1,
 'baths_half': 0,
 'baths_1qtr': 0,
 'year_built': 1951,
 'lot_sqft': 6030,
 'sqft': 2466,
 'garage': 1,
 'stories': 1,
 'beds': 4,
 'tags': ['community_outdoor_space',
  'community_security_features',
  'dining_room',
  'dishwasher',
  'family_room',
  'fireplace',
  'forced_air',
  'hardwood_floors',
  'ocean_view',
  'view',
  'washer_dryer',
  'water_view',
  'basement',
  'garage_1_or_more'],
 'new_construction': False,
 'distance_to_home': 8.219434861515992,
 'angle_from_home': 1.490368859537127}

In [38]:
# This will randomly take 10% of the data as a validation set.
test_indicies = random.sample(range(0,len(fg.targets)), int(0.1 * len(fg.targets)))
train_indicies = set(range(0, len(fg.targets))).difference(set(test_indicies))

train_features = list(map(fg.features.__getitem__, train_indicies))
train_targets = list(map(fg.targets.__getitem__, train_indicies))
test_features = list(map(fg.features.__getitem__, test_indicies))
test_targets = list(map(fg.targets.__getitem__, test_indicies))

In [64]:
'''
Days Listed - Linear
Days Updated - Linear
*baths - Normalize
year_built - I want to bucketize these, then keep the dummies.
lot_sqft - Normalize
    Feature generation, multiply lot_sqft normalized and the inverse_distance
        More questions, can you do this?
sqft - Normalize
garage / stories / beds - Normalize
Tags - Do what we did in the nlp homework
'''

from sklearn.preprocessing import MinMaxScaler, StandardScaler, KBinsDiscretizer, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error

from collections import Counter

class ToDataFrame(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return pd.DataFrame(X).drop_duplicates(subset=['Property_ID'])

class DictEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.tags_to_keep = None

    def fit(self, X, y=None):
        tag_frequency = Counter(chain(*X))
        self.tags_to_keep = [k for k, v in tag_frequency.items() if v > 1]
        return self
    
    def transform(self, X):
        return pd.Series(map(lambda l : {k : 1 for k in l if l in self.tags_to_keep}, X))

minmax_cols = ['Days_listed', 'Days_listed', 'distance_to_home', 'angle_from_home']
normalize_cols = ['lot_sqft', 'sqft']
bucketize_cols = ['year_built']
dummy_cols = ['baths_full', 'baths_full', 'baths_full', 'baths_1qtr', 'garage', 'stories', 'beds']

preprocess_tags_col = Pipeline(
    [
        ('dict_encode', DictEncoder()),
        ('dict_vectorize', DictVectorizer())
    ]
)

preprocess_data = ColumnTransformer(
    [
        ('scale', MinMaxScaler(), minmax_cols),
        ('normalize', StandardScaler(), normalize_cols),
        ('bucketize', KBinsDiscretizer(), bucketize_cols),
        ('dummy', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), dummy_cols),
        ('list', preprocess_tags_col, 'tags')
    ]
)

gbr_pipeline = Pipeline(
    [
        ('to_data_frame', ToDataFrame()),
        ('preprocess', preprocess_data),
        ('gbr', GradientBoostingRegressor(n_estimators=100))
    ]
)

grid = {
    'gbr__loss' : ['squared_error', 'absolute_error'],
    'gbr__max_depth' : [5, 6, 7]
}

#best_gbr = GridSearchCV(gbr_pipeline, param_grid=grid, cv=5, verbose=10, n_jobs=4).fit(fg.features, fg.targets)
#print(f'best params {best_gbr.best_params_}')
#print(f'mean absolute error: {mean_absolute_error(best_gbr.predict(fg.features), fg.targets)}')

In [172]:
import xgboost as xgb

xgb_pipeline = Pipeline(
    [
        ('to_data_frame', ToDataFrame()),
        ('preprocess', preprocess_data),
        ('xgb', xgb.XGBRegressor(objective ='reg:pseudohubererror'))
    ]
)

grid = {
    'xgb__n_estimators' : [50, 75, 100],
    'xgb__max_depth' : [5, 6, 7]
}

#best_xgb = GridSearchCV(xgb_pipeline, param_grid=grid, cv=5, verbose=0, n_jobs=4).fit(fg.features, fg.targets)
#print(f'best params {best_xgb.best_params_}')
#print(f'mean absolute error: {mean_absolute_error(best_xgb.predict(fg.features), fg.targets)}')

best params {'xgb__max_depth': 5, 'xgb__n_estimators': 50}
mean absolute error: 1060192.4117948718


In [165]:
from sklearn.linear_model import Ridge

ridge_pipeline = Pipeline(
    [
        ('to_data_frame', ToDataFrame()),
        ('preprocess', preprocess_data),
        ('ridge', Ridge())
    ]
)

grid = {
    'ridge__alpha' : [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 1e2, 1e3, 1e4, 1e5]
}

#best_rigdge = GridSearchCV(ridge_pipeline, param_grid=grid, cv=5, verbose=0, n_jobs=4).fit(fg.features, fg.targets)
#print(f'best params {best_rigdge.best_params_}')
#print(f'mean absolute error: {mean_absolute_error(best_rigdge.predict(fg.features), fg.targets)}')

best params {'ridge__alpha': 100.0}
mean absolute error: 264198.579836482


In [60]:
# best_xgb.best_estimator_.named_steps['preprocess'].named_transformers_['list'].named_steps['dict_vectorize'].get_feature_names_out()[:3]
# len(best_xgb.best_estimator_.named_steps['xgb'].feature_importances_)

In [126]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.pipeline import FunctionTransformer, FeatureUnion
from scikeras.wrappers import KerasRegressor

class generate_keras_model(BaseEstimator, RegressorMixin):
    def __init__(self, first_layer = None):
        self.first_layer = first_layer

    def _keras_model(self, n_cols):
        km = Sequential()
        km.add(Dense(self.first_layer, input_shape=(n_cols,), activation='relu', name='dense_1'))
        km.add(Dense(64, activation='relu', name='dense_2'))
        km.add(Dense(1, activation='linear', name='dense_output'))
        km.compile(optimizer='adam', loss='mse', metrics=['mae'])
        return KerasRegressor(model=km)

    def fit(self, X, y=None):
        self.model = self._keras_model(n_cols=X.shape[1])
        self.model.fit(X, y)
        return self

    def predict(self, X):
        return self.model.predict(X)

keras_pipeline = Pipeline(
    [
        ('to_data_frame', ToDataFrame()),
        ('preprocess', preprocess_data),
        ('keras_model', generate_keras_model())
    ]
)

grid = {
    'keras_model__first_layer' : [128, 256]
}

best_keras = GridSearchCV(
    keras_pipeline, 
    param_grid=grid, 
    cv=5, 
    verbose=1, 
    n_jobs=4,
    scoring='neg_mean_absolute_error'
).fit(train_features, train_targets)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
51/51 [==============================] - 1s 5ms/step - loss: 1527987699712.0000 - mae: 1081912.7500


In [128]:
best_keras.best_estimator_

Pipeline(steps=[('to_data_frame', ToDataFrame()),
                ('preprocess',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  ['Days_listed', 'Days_listed',
                                                   'distance_to_home',
                                                   'angle_from_home']),
                                                 ('normalize', StandardScaler(),
                                                  ['lot_sqft', 'sqft']),
                                                 ('bucketize',
                                                  KBinsDiscretizer(),
                                                  ['year_built']),
                                                 ('dummy',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['baths_full', 'baths_full',
                                                   'baths_full', 'baths_1qtr',
                                                   'garage', 'stories',
                                                   'beds']),
                                                 ('list',
                                                  Pipeline(steps=[('dict_encode',
                                                                   DictEncoder()),
                                                                  ('dict_vectorize',
                                                                   DictVectorizer())]),
                                                  'tags')])),
                ('keras_model', generate_keras_model(first_layer=256))])

In [69]:
test_pipeline = Pipeline(
    [
        ('to_data_frame', ToDataFrame()),
        ('preprocess', preprocess_data)
     ]
)

xx = pd.DataFrame(test_pipeline.fit_transform(train_features))[0]